In [ ]:
from requests import get
from bs4 import BeautifulSoup as bs
import pandas as pd

url = "https://www.ticketlink.co.kr/ranking?ranking=genre&categoryId=10&category2Id=14&category3Id=14&period=yearly&currentDate=1733792490744"

res = get(url)
res.raise_for_status()

all_info = bs(res.text, 'html.parser')

concerts = []

# 각 공연의 링크 추출
for link in all_info.select('a[class*="ranking-item"]'):
    href = link.get('href')
    if href and '/product/' in href:
        detail_url = f"https://www.ticketlink.co.kr{href}"
        
        # 상세 페이지 요청
        detail_response = get(detail_url)
        detail_response.raise_for_status()
        detail_soup = bs(detail_response.text, 'html.parser')
        
        # 공연명 추출
        title = detail_soup.select_one('h1.product-title').get_text(strip=True)
        
        # 장소 추출
        place = detail_soup.select_one('div.place-info').get_text(strip=True)
        
        # 가격 추출
        price = detail_soup.select_one('div.price-info').get_text(strip=True)
        
        # 공연 정보 저장
        concerts.append({
            '공연명': title,
            '장소': place,
            '가격': price,
            '상세페이지': detail_url
        })

# 데이터프레임 생성
df = pd.DataFrame(concerts)

df